In [1]:
import os
import openai
from dotenv import load_dotenv
from sql_market_agent.agent.agent import create_sql_market_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain.callbacks import StdOutCallbackHandler
import matplotlib.pyplot as plt

load_dotenv()

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
DB_HOST = os.getenv("POSTGRES_HOST", "localhost")
DB_PORT = os.getenv("POSTGRES_PORT", "5432")
DB_NAME = os.environ.get("POSTGRES_DB")
DB_USER = os.environ.get("POSTGRES_USER")
DB_PASSWORD = os.environ.get("POSTGRES_PASSWORD")

2024-01-29 16:11:14,791 - INFO - NumExpr defaulting to 8 threads.


In [2]:
%matplotlib inline


In [3]:
openai.api_key = OPENAI_API_KEY


llm = ChatOpenAI(temperature=0, model="gpt-4", streaming=True)

# Create postgres connection string:
# db_connection_string = (
#     f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
# )
agent_executor = create_sql_market_agent(
    llm=llm,
    # db_connection_string=db_connection_string,
    preinitialize_database=True,
    stocks=["CAT", "NKE", "XOM"],
)

2024-01-29 16:11:17,915 - INFO - DB connection string not provided, using local db on disc...
2024-01-29 16:11:17,928 - INFO - Initializing database with StockData table...
2024-01-29 16:11:17,945 - INFO - Database (sqlite) initialized.
2024-01-29 16:11:17,946 - INFO - Fetching data for analysis...
/home/oleh/arkad/arkad-demo/arkad-sql-demo/notebooks/.venv/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/home/oleh/arkad/arkad-demo/arkad-sql-demo/notebooks/.venv/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/home/oleh/arkad/arkad-demo/arkad-sql-demo/notebooks/.venv/lib/python3

In [5]:
response = agent_executor(
            {"input": "plot close price for XOM most recent 5 days", "chat_history": []},
            # callbacks=[StdOutCallbackHandler()],
)



> Entering new AgentExecutor chain...


2024-01-29 16:13:13,336 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `sql_database_tool` with `What is the most recent date for which you have data in the database?`




> Entering new AgentExecutor chain...


2024-01-29 16:13:15,282 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


To find the most recent date in the database, I need to identify the tables and their schemas first. Then, I can look for fields that likely contain date information, such as those named 'date', 'timestamp', etc. After identifying these fields, I can write a SQL query to find the most recent date.
Action: sql_db_list_tables
Action Input: ""stockdata

...

The SQL query is correct. Now I can execute it to find the most recent date in the 'stockdata' table.
Action: sql_db_query
Action Input: "SELECT MAX(date) FROM stockdata"[('2024-01-26',)]

2024-01-29 16:13:30,232 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


I now know the final answer.
Final Answer: The most recent date for which we have data in the database is 2024-01-26.

> Finished chain.
The most recent date for which we have data in the database is 2024-01-26.

2024-01-29 16:13:32,177 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The most recent date for which we have data in the database is January 26, 2024.

> Finished chain.
